# 무관심 가사 20개 Markov Chain 하기

In [4]:
import os
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random

file = open("../../Data/ballad_lyrics_indf.csv",encoding='UTF-8')

# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        try:
            w3 = word_choice(dic[w1][w2])
            ret.append(w3)
        except:
            continue
        # if w3 == ".": break
        if len(ret) >= 200: break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    # 네이버 맞춤법 검사기를 사용합니다.
    # data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?" + params)
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

# 문장 읽어 들이기 --- (※4)
toji_file = str(file.readlines())
dict_file = "../../Note/Markov/markov_ballad_sad.json"
if not os.path.exists(dict_file):
    # 텍스트 파일 읽어 들이기
    with open(toji_file, 'r', encoding='utf-8') as f:
        text = str(f.readlines())

    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    text = text.replace("n", "")
    # 형태소 분석
   # twitter = Twitter()
    twitter=Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))

# 문장 만들기 — (※6)
for i in range(5):
    s = make_sentence(dic)
    print(s)
    print("—")

떨어질 줄 몰랐던 너의 품도 잡은 손에 스치던 봄바람도 떨리던 입맞춤도 맞춰 걷던 걸음도 내겐 모두 처음이라서 유난히 아픈가 봐 시간이 지나면 널 보며 웃는 날 언젠가 오겠지 그때까지만 더 아파해볼 게 다 부담이 돼 그렇게 내게 말하지 못했어 좋은 기억마저 사라질까 봐 안녕 내 전부였던 너 사랑해 너라서 유난히 좋았나 봐 시간이 지나면 널 보며 웃는 날 언젠가 오겠지 그때까지만 더 아파해볼게 누군가를 위한 노랠 부른 것도 벚꽃과 여름바다 단풍과 함박눈이 처음이라서 고된 하루 끝을 미뤘나 봐 나의 첫 사람이네 첫사랑이 너의 추억들은 아팠고 네가 보고 싶다 n 자꾸만 연락을 피하고 바쁘다며 약속을 미루고 점점 변해가는 네 모습 웃고 있는 넌 잘 지냈나 봐 찾아가지 말걸 그랬어 믿고 싶었는데 영화 같은 일 내겐 없나 봐 나를 다 잊은 척 웃어도 보고 드라마처럼 혼자 취해도 보고 널 잊으려고 하면 할수록 너무 보고 싶어나 술에 잊자 그만 나도 숨 좀 쉬자 제발 마치 주문처럼 매일 읊조렸던 넌 모르는 내 슬픈 혼잣말 친구들과 통화에
—
떨어질 줄 몰랐던 너의 품도 잡은 손에 스치던 봄바람도 떨리던 입맞춤도 맞춰 걷던 걸음도 내겐 모두 처음이라서 힘든가 봐 나의 이별 끝에 아직도 서있어 지금도 네게 이런 마음이 너무 두려워 너의 목소리 마치 들린 것 같아 오늘까지만 그리워할 게 끝이 보이지 않아내 사랑 n 하루만 더 참을 순 없었니 한번 만나 이렇게 널 불러볼게 사실은 겁이 나 이젠 끝나가지만 함께한 기억은 가시가 됐고다 사라졌으면 해 모두 지워졌으면 해나 쉴 수 있게 널 떠날게 가질 수 없는 상처로 남아 내겐 그 모든 것이 내겐 부담이 돼 그렇게 내게 말하지 못했어 좋은 기억마저 사라질까 봐 안녕 내 사랑은 갔고 남겨진 추억들은 없어 마치 우린 없었던 사이 n 혼자 이별을 하고 가버리면 난 어떻게 하라고 너 없는 밤 아프게 지워가지만 내 맘이 너무 아파서 널 보면 나와달라 아직도 난 널 잊고 행복하게 잘 살았는데 예쁜 사랑 다시 시작했는데 다 잊었다 생각하고 지내 왔는